### Set up the Spark environment.

In [ ]:
import sys
sys.path.append('/class')
from initspark import *
sc, spark, conf = initspark()


### Turn a simple RDD into a DataFrame. 

In [ ]:
x = sc.parallelize([(1,'alpha'),(2,'beta')])
print(x.collect())
x0 = spark.createDataFrame(x)
x0.show()
x0.collect()

### Give the DataFrame meaningful column names, by providing a list of names as string.

In [ ]:
x1 = spark.createDataFrame(x, schema=['ID','Name'])
x1.show()
print(x1)
print(x1.collect())
x2 = x1.take(1)
print(x2[0])
print(x2[0].ID, x2[0]['Name'])

### Give a DataFrame a schema with column names and data types, by providing a single string with the names and data types.

In [ ]:
x2 = spark.createDataFrame(x, schema = 'ID:int, Name:string')
x2.show()
print(x2)


### Create a schema object to be more specific. Also some functions cannot use string but require schema objects.

In [ ]:
# CREATE TABLE schema1 (ID int, Name string);
schema1 = StructType([
    StructField('ID', IntegerType()), 
    StructField('Name', StringType())
])
x3 = spark.createDataFrame(x, schema = schema1)
x3.show()
print(x3)


### The built in toDF method does the same thing.

In [ ]:
x.toDF().printSchema()
x.toDF(['ID', 'Name']).printSchema()
x.toDF('ID:int, Name:string').printSchema()
x.toDF(schema = schema1).printSchema()


## LAB: ## 
### Use the regions and territories RDDs from the previous lab and convert them into DataFrames with meaningful schemas.
<br>
<details><summary>Click for <b>hint</b></summary>
<p>
Use sc.textFile to read the files
<br>
Use map functions to split and convert the data
<br>
Use spark.createDataFrame and toDF to convert RDD into DataFrames
<br>
<br>
</p>
</details>

<details><summary>Click for <b>code</b></summary>
<p>

```python
regions = sc.textFile('hdfs://localhost:9000/regions')
regions = regions.map(lambda x : x.split(',')).map(lambda x : (int(x[0]), x[1]))
regionsdf = spark.createDataFrame(regions, 'RegionID:int, RegionName:string')
regionsdf.show()

territories = sc.textFile('hdfs://localhost:9000/territories')
territories = territories.map(lambda x : x.split(',')).map(lambda x : (int(x[0]), x[1], int(x[2])))
territoriesdf = territories.toDF('TerritoryID:int, TerritoryName:string, RegionID: int')
territoriesdf.show()

# instead of collecting the data as a list of rows, it can be brought back as a pandas DataFrame
regions.toPandas()
```
</p>
</details>

### Examples of reading a CSV directly into a DataFrame.

### If there is a top level read function for the file type you want, that's the cleanest option, and pass in the parameters as named parameters. Not all formats have this and also legacy code written before this may use the old style syntax.

In [ ]:
filename1 = 'file:///class/datasets/northwind/CSVHeaders/categories'
cat1 = spark.read.csv(filename1, header = True, inferSchema = True)
print(cat1)
cat1.printSchema()
cat1.show()
cat1.collect()
# CREATE TEMPORARY TABLE categories (CategoryID int, CategoryName string, Description string)
# ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
# LOCATION 'file:///class/datasets/northwind/CSVHeaders/categories'

In [ ]:
display(cat1)


### Legacy style is still seen and uses a few alternate syntaxes.

In [ ]:
filename2 = 'file:///class/datasets/northwind/TSV/categories'
cat2 = spark.read.load(filename2, format = 'csv', sep = '\t', inferSchema = True, header = False)
cat2.printSchema()
cat2.show()

In [ ]:
filename3 = 'file:///class/datasets/northwind/CSVHeaders/categories'
cat3 = spark.read.load(filename3, format = 'csv', sep = ',', inferSchema = True, header = True)
cat3.printSchema()
cat3.show()


### There are several alternate syntaxes which can be confusing, but since you will encounter them, you need to learn to recognize the different options
option and options allow you pass parameters in different ways, but not the true is quoted and lowercase because it is a java value, but you could also pass it as a True Python value.

In [ ]:
cat4 = spark.read.format('csv').option('header','true').option('inferSchema','true').load(filename1)
cat4.printSchema()
cat5 = spark.read.format('csv').options(header=True, inferSchema='true').load(filename1)
cat5.printSchema()


### As the tables get more complex, there is a Jupyter command that will show the tables in a prettier format.

In [ ]:
display(cat3)

## LAB: ## 
### Load the products table using any of the spark.read methods.
<br>
<details><summary>Click for <b>hint</b></summary>
<p>
Use spark.read.csv
<br>
Make sure to read the version that has headers if you want to infer schema
<br><br>
</p>
</details>

<details><summary>Click for <b>code</b></summary>
<p>

```python
prod1 = spark.read.csv('file:///class/datasets/northwind/CSVHeaders/products', header=True, inferSchema=True)
prod1.printSchema()
display(prod1)```
</p>
</details>

In [ ]:
#prod1 = spark.read.csv('file:///class/datasets/northwind/CSVHeaders/products', header=True, inferSchema=True)
#prod1 = spark.read.json('file:///class/datasets/northwind/JSON/products')
prod1 = spark.read.orc('hdfs://localhost:9000/products')
prod1.printSchema()
display(prod1)

### Using a schema is a good idea for performance if you know what it is. Usually you can infer schema during development and use it as a helper to build the schema to use for production.

In [ ]:
#  |-- productid: short (nullable = true)
#  |-- productname: string (nullable = true)
#  |-- supplierid: short (nullable = true)
#  |-- categoryid: short (nullable = true)
#  |-- quantityperunit: string (nullable = true)
#  |-- unitprice: decimal(10,0) (nullable = true)
#  |-- unitsinstock: short (nullable = true)
#  |-- unitsonorder: short (nullable = true)
#  |-- reorderlevel: short (nullable = true)
#  |-- discontinued: integer (nullable = true)

prodSchema = StructType([
    StructField('ProductID', IntegerType()), 
    StructField('ProductName', StringType()),
    StructField('SupplierID', IntegerType()), 
    StructField('CategoryID', IntegerType()), 
    StructField('QuantityPerUnit', StringType()), 
    StructField('UnitPrice', FloatType()), 
    StructField('UnitsInStock', IntegerType()), 
    StructField('UnitsOnOrder', IntegerType()), 
    StructField('ReorderLevel', IntegerType()), 
    StructField('Discontinued', IntegerType())
])

prod2 = spark.read.csv('file:///class/datasets/northwind/CSVHeaders/products', header=True, schema=prodSchema)
#prod2 = spark.read.json('file:///class/datasets/northwind/JSON/products', schema=prodSchema)
print(prod2)
display(prod2)

# prod2 = spark.read.csv('file:///class/datasets/northwind/CSVHeaders/products', header=True, inferSchema=False)
# print(prod2)
# prodSchema2 = "ProductID:int, ProductName:string, SupplierID:int, CategoryID:int, QuantityPerUnit:string, UnitPrice:double, UnitsInStock:int, UnitsOnOrder:int, ReorderLevel:int, Discontinued:int"

# prod3 = prod2.toDF(prodSchema2)


### Convert a DataFrame into a JSON string.

In [ ]:
print (cat1.toJSON().take(10))

### Or you can chain the whole thing together into a single command.

In [ ]:
j = spark.read.csv('file:///class/datasets/northwind/CSVHeaders/categories', inferSchema=True, header=True).toJSON().collect()
print(j)

### JSON is another top level supported format.

In [ ]:
cat6 = spark.read.json('file:///class/datasets/northwind/JSON/categories')
display(cat6)
print(cat6)

### You can also use schemas but be careful of case because it matches the schema names to the key names in the JSON and upper/lower case matters. This will fail.

In [ ]:
prod = spark.read.json('file:///class/datasets/northwind/JSON/products', schema=prodSchema)
display(prod)



### Redefine the schema using all lower case since that's what this particular JSON file has.

In [ ]:
prodSchema = StructType([
    StructField('productid', IntegerType()), 
    StructField('productname', StringType()),
    StructField('supplierid', IntegerType()), 
    StructField('categoryid', IntegerType()), 
    StructField('quantityperunit', StringType()), 
    StructField('unitprice', FloatType()), 
    StructField('unitsinstock', IntegerType()), 
    StructField('unitsonorder', IntegerType()), 
    StructField('reorderlevel', IntegerType()), 
#    StructField('joey', StringType()),
    StructField('discontinued', IntegerType())
])

prod = spark.read.json('file:///class/datasets/northwind/JSON/products', schema=prodSchema)
print(prod)
display(prod)


### You may also see the older style syntax.

In [ ]:
prod = spark.read.format('json').load('file:///class/datasets/northwind/JSON/products')
display(prod)


### Writing a DataFrame uses a similar syntax. There is a safeguard against accidentally overwriting a destination, so that's why we are deleting it first.

In [ ]:
! rm -r /tmp/prodjson
prod.write.json('file:///tmp/prodjson')
#! cat /tmp/prodjson/*

### Alternatively generate a unique filename with a timestamp.

In [ ]:
import time

prod.write.csv(f'file:///tmp/prodcsv{time.strftime("%Y%m%d-%H%M%S")}', sep = '|', header=True)
! cat /tmp/prodcsv*/*
! ls /tmp/prodcsv*

### Note the use of mode('overwrite') here as an alternative to deleting it first.

In [ ]:
prod.write.mode('overwrite').orc('file:///tmp/prodorc')
! cat /tmp/prodorc/*


In [ ]:
prod.write.mode('overwrite').parquet('file:///tmp/prodparquet')
! cat /tmp/prodparquet/*


### AVRO is a little different, it is built in now but doesn't have a top level method for it, so you need to use the old style syntax.
This doesn't always work inside of a notebook either, so take a look at the program and run it from spark-submit with the proper package dependency added.

In [ ]:
! cat /class/avro.py

! spark-submit --packages org.apache.spark:spark-avro_2.11:2.4.3 /class/avro.py
        
#prod4.write.format("avro").mode('overwrite').save('/tmp/prodavro')


## LAB: ## 
### Try to read in a few files with different formats and write them out to other formats. 
### Read shippers found in TSV and write it out as JSON.
### Read orders found in CSVHeaders and write it out as ORC.
### Read orderdetails found in JSON and write it out as Parquet.
<br>
<details><summary>Click for <b>hint</b></summary>
<p>
Use the syntax that is easiest for you
<br>
If there is a top level function for the file format, that is usually the easiest way
<br>
TSV is just the same as CSV, but if there are no headers you need to supply a schema
<br>
Remember to either remove the destination folder before writing or use an overwrite option
<br><br>
</p>
</details>

<details><summary>Click for <b>code</b></summary>
<p>

```python
shipperSchema = StructType([
    StructField('shipperid', StringType()), 
    StructField('shippername', StringType()),
    StructField('phone', StringType())
])

shippers = spark.read.csv('file:///class/datasets/northwind/TSV/shippers', sep='\t', header=False, inferSchema=False, schema=shipperSchema)
shippers.write.mode('overwrite').json('file:///tmp/shippersjson')

orders = spark.read.csv('file:///class/datasets/northwind/CSVHeaders/orders', header=True, inferSchema=True)
orders.write.mode('overwrite').orc('file:///tmp/ordersorc')

orderdetails = spark.read.json('file:///class/datasets/northwind/JSON/orderdetails')
orderdetails.write.mode('overwrite').parquet('file:///tmp/orderdetailsparquet')

```
</p>
</details>

### Can start to call methods on the DataFrame.

In [ ]:
prod.printSchema()
print (prod.columns, prod.count())

### Choose particular columns from a DataFrame.
You can use quoted strings for the column names

In [ ]:
display(prod.select('productid', 'productname', 'unitprice'))

### Or you can use a pythonic syntax using the DataFrame name and field name

In [ ]:
display(prod.select(prod.productid, prod.productname, prod.unitprice, 'categoryid'))

fields = ['categoryid', 'productid']
display(prod.select(fields))

### Case is ignored if you use quoted strings but not if you use python syntax

In [ ]:
display(prod.select('Productid', 'productname', 'unitprice'))

# this will fail
#display(prod.select(prod.Productid, prod.productname, prod.unitprice))

# this will not fail
display(prod.select(prod.productid, prod.productname, prod.unitprice))

### `distinct` is a method after the `select` method chooses the columns

In [ ]:
display(prod.select('CategoryID').distinct())

# SELECT DISTINCT categoryid from prod
# SELECT categoryID from prod GROUP BY categoryid
# FROM prod SELECT categoryID DISTINCT

### Sort a DataFrame. The `sort` and `orderBy` methods are different aliases for the exact same method.

In [ ]:
field = 'productid'
# display(prod.sort(prod.unitprice))
# display(prod.orderBy('unitprice', ascending = False))
display(prod.select(field, 'productname', prod.unitprice).orderBy('unitprice'))

### Create a new DataFrame with a new calculated column added.

In [ ]:
prod2 = (prod.withColumn('value', prod.unitprice * prod.unitsinstock)
             .withColumn('other', prod.unitprice * 10))
display(prod2)


### Remove an unwanted column from a DataFrame.

In [ ]:
print(prod.columns)
prod.select('categoryid',
 'discontinued',
 'productid',
 'productname',
 'reorderlevel',
 'supplierid',
 'unitprice',
 'unitsinstock',
 'unitsonorder')

### `drop` is a convenient way to remove a column without enumerating through all the columns you want to keep.

In [ ]:
prod2 = prod2.drop('quantityperunit')
display(prod2)
# SELECT * EXCEPT(col1, col2)


### The `filter` and `where` methods can both be used and have alternative ways to represent the condition.

In [ ]:
p = prod
display(p.filter(p.unitprice > 100))
display(p.filter('unitprice > 100'))

# Note == when using python syntax
display(p.where(p.categoryid == 2))

# Note = when using quoted SQL like syntax
display(p.where('categoryid = 2'))



### More complex conditions

In [ ]:
display(p.where('unitprice >= 50 and unitprice <= 100'))
display(p.where('unitprice between 50 and 100'))

display(p.where((p.unitprice >=50) and (p.unitprice <= 100)))

# fails
# display(p.where((p.unitprice >=50) and (p.unitprice <= 100)))


## LAB: ## 
### Find all the products in category 2 with fewer units in stock than units on order 
### Only display with productid, name, unitsinstock, unitsonorder and unitprice
<br>
<details><summary>Click for <b>hint</b></summary>
<p>
Use the where or filter method. It's probably easier to use a quoted SQL style syntax
<br>
Use select to get the columns you want to see
<br><br>
</p>
</details>

<details><summary>Click for <b>code</b></summary>
<p>

```python
display(p.where('unitsinstock < unitsonorder and categoryid = 2')
         .select('productid','productname', 'unitsinstock', 'unitsonorder', 'unitprice'))
```
</p>
</details>

In [ ]:
display(p.where('unitsinstock < unitsonorder and categoryid = 2')
         .select('productid','productname', 'unitsinstock', 'unitsonorder', 'unitprice'))


### JOINs work as expected.

In [ ]:
tab1 = sc.parallelize([(1, 'Alpha'), (2, 'Beta'), (3, 'Delta')]).toDF('ID:int, code:string')
tab2 = sc.parallelize([(100, 'One', 1), (101, 'Two', 2), (102, 'Three', 1), (103, 'Four', 4)]).toDF('ID:int, name:string, parentID:int')
# tab1.join(tab2, tab1.ID == tab2.parentID).show()
# tab1.join(tab2, tab1.ID == tab2.parentID, 'left').show()
# tab1.join(tab2, tab1.ID == tab2.parentID, 'right').show()
# tab1.join(tab2, tab1.ID == tab2.parentID, 'full').show()


###  Examples of aggregate functions.

In [ ]:
tab3 = sc.parallelize([(1, 10), (1, 20), (1, 30), (2, 40), (2,50)]).toDF('groupID:int, amount:int')
tab3.groupby('groupID').max().show()
#tab3.groupby('groupID').sum().drop('sum(groupID)').show()


### Alternatively there is a `agg` method that takes a `dict` object.

In [ ]:
x = tab3.groupby('groupID')
x.agg({'amount':'sum'}).show()


### Or we could import the functions from pyspark and use them directly. This allows us to call multiple aggregates at once.

In [ ]:
from pyspark.sql import functions as F
#from pyspark.sql.functions import sum, max
x.agg(F.sum('amount'), F.max('amount')).show()
#dir(F)

### Or we could use SQL syntax to encode the calculations but we need to use the `expr` function to tell it how to interpret the SQL.

In [ ]:
from pyspark.sql.functions import expr
f = expr('sum(amount) as total')
print(f, type(f))
x.agg(expr('sum(amount) as total'), expr('count(*) as cnt')).show()


## LAB: ## 
### Join products and categories together displaying only the product and category ID's and names, sort by categoryid and productid
<br>
<details><summary>Click for <b>hint</b></summary>
<p>
Make sure not to show the common column twice
<br>
Select with python style makes it easier to distinguish which columns you want from a join
<br><br>
</p>
</details>

<details><summary>Click for <b>code</b></summary>
<p>

```python
c = spark.read.csv('file:///class/datasets/northwind/CSVHeaders/categories'
                   , inferSchema = True, header=True)
p = spark.read.json('file:///class/datasets/northwind/JSON/products')

# display(c)
j = (c.join(p, c.CategoryID == p.categoryid)
      .select(c.CategoryID, c.CategoryName, p.productid, p.productname)
      .orderBy('categoryid', 'productid'))
```
</p>
</details>

### Can also convert a DataFram back to and RDD if we want to use low level RDD methods.

In [ ]:
j1 = j.rdd
print(j.rdd.map(lambda x : (x.CategoryID, x.productid)).toDF(['x','y']).take(2))

### Sometimes you want to just rename a column so here are two ways to accomplish that

In [ ]:
display(p.withColumnRenamed('unitprice','listprice'))

cols = p.columns # get a list of all the current column names
cols[5] = 'listprice' # replace a column position with the new name 
p1 = p.toDF(*cols) # create a new dataframe from the original with a list of column names
display(p1)


## HOMEWORK: ## 
### Join Orders, OrderDetails and Products together. Find the sales total for each category listed in descending order by sales
<br>
<details><summary>Click for <b>hint</b></summary>
<p>
Load each file into a dataframe and give them single letter aliases for simplicity
<br>
Join products and order details together on productid
<br>
Join that to orders on orderid
<br>
Createa a calculated column to get the line total for each order details
<br>
Group by categoryID and calculate the sum of the line totals 
<br>
Sort on the calculated total
<br><br>
</p>
</details>

<details><summary>Click for <b>code</b></summary>
<p>

```python
prodSchema = StructType([
    StructField('productid', IntegerType()), 
    StructField('productname', StringType()),
    StructField('supplierid', IntegerType()), 
    StructField('categoryid', IntegerType()), 
    StructField('quantityperunit', StringType()), 
    StructField('unitprice', FloatType()), 
    StructField('unitsinstock', IntegerType()), 
    StructField('unitsonorder', IntegerType()), 
    StructField('reorderlevel', IntegerType()), 
    StructField('discontinued', IntegerType())
])


```
</p>
</details>
